# Hands-on Python Excersises:

- Tutorial 0 : Getting Started with Movingpandas
- ***Tutorial 1 : Trajectory Preprocessing***
- Tutorial 2 : Taxi Trajectory Dashboard Visualization



# Tutorial 1 : Trajectory Preprocessing

## Trajectory Preprocessing Recap

Trajectory data often contains various challenges like noise, redundancy, or uninformative segments. Preprocessing helps to clean and optimize the data for further analysis or visualization. Below is a brief overview of some key trajectory preprocessing techniques:

1. **Trajectory Noise Filtering**
- Remove irregularities or noise from raw trajectory data to reveal underlying movement patterns.
2. **Stay Point Detection**
- Identify locations where a moving object remains stationary or within a small area for a defined period. These "stay points" often indicate points of interest like stops or dwell locations.
3. **Trajectory Segmentation (Splitting Trajectories)**
- Divide long or complex trajectories into smaller, meaningful segments to better analyze different movement behaviors.
4. **Trajectory Compression (Generalization)**
- Reduce the number of points in a trajectory while preserving the overall shape and important characteristics.


```
! pip install movingpandas

! pip install cartopy

! pip install geoviews

! pip install mapclassify

! pip install keplergl
```


In [ ]:
# ! pip install movingpandas
# ! pip install cartopy
# ! pip install geoviews
# ! pip install mapclassify
# ! pip install keplergl

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import matplotlib.pyplot as plt

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta

import folium

import warnings

warnings.filterwarnings("ignore")


## Noise Filtering


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/i4Geo_Workshop/new_taxi_porto.csv"
taxi_data = pd.read_csv(path)
taxi_data = taxi_data.head(5000)
taxi_data.head()

In [ ]:
traj_collection = mpd.TrajectoryCollection(taxi_data, "TRIP_ID", t="datetime", y="latitude", x="longitude", crs=4326)
print(traj_collection)

In [ ]:
my_traj = traj_collection.trajectories[60]

In [ ]:
m = my_traj.explore(style_kwds={'weight': 5, 'color': 'blue'}, name="My trajectory")
my_traj.df.explore(style_kwds={'weight': 10, 'color': 'red'}, m=m, name="My trajectory points")
#add a layer control to the map
folium.LayerControl().add_to(m)
m

Let's show the speed information too

In [ ]:
m = my_traj.explore(column="speed", style_kwds={'weight': 10})
my_traj.df.explore(style_kwds={'weight': 10, 'color': 'red'}, m=m)

One easy way to remove noises, is to calculate the speed and flag the abnormal high speeds

In [ ]:
smooth_traj = mpd.OutlierCleaner(my_traj).clean(v_max=100, units=("km", "h"))
print(smooth_traj)

In [ ]:
m = smooth_traj.explore(column="speed", style_kwds={'weight': 10})
smooth_traj.df.explore(style_kwds={'weight': 10, 'color': 'red'}, m=m)

## Detecting Stops

In [ ]:
my_traj = traj_collection.trajectories[23]
m = my_traj.explore(style_kwds={'weight': 5, 'color': 'blue'}, name="My trajectory")
my_traj.df.explore(style_kwds={'weight': 10, 'color': 'red'}, m=m, name="My trajectory points")
#add a layer control to the map
folium.LayerControl().add_to(m)
m

In [ ]:
detector = mpd.TrajectoryStopDetector(my_traj)

In [ ]:
stop_time_ranges = detector.get_stop_time_ranges(
    min_duration=timedelta(seconds=60), max_diameter=100
)

In [ ]:
for x in stop_time_ranges:
    print(x)

In [ ]:
stop_points = detector.get_stop_points(
    min_duration=timedelta(seconds=60), max_diameter=100
)

In [ ]:
stop_points

Let's show it on the map

In [ ]:
# Create a GeoDataFrame from the stop points
stop_points_gdf = gpd.GeoDataFrame(stop_points, geometry="geometry", crs="EPSG:4326")

m = my_traj.explore(
    color="blue",
    style_kwds={"weight": 4},
    name="Trajectory"
)

my_traj.df.explore(color="red", style_kwds={'weight': 5}, m=m, name="Trajectory points")

stop_points_gdf.explore(
    m=m,
    color="red",
    style_kwds={
        "style_function": lambda x: {"radius": x["properties"]["duration_s"]/4}
    },
    name="Stop points",
)

folium.TileLayer("OpenStreetMap").add_to(m)
folium.LayerControl().add_to(m)

m

Stop Detection for TrajectoryCollections

In [ ]:
detector = mpd.TrajectoryStopDetector(traj_collection)
stop_points = detector.get_stop_points(
    min_duration=timedelta(seconds=120), max_diameter=100
)
len(stop_points)

In [ ]:
ax = traj_collection.plot(figsize=(7, 7))
stop_points.plot(ax=ax, color="red")

## Trajectory Segmentation

In [ ]:
traj_collection = mpd.TrajectoryCollection(taxi_data, "TAXI_ID", t="datetime", y="latitude", x="longitude", crs=4326)
print(traj_collection)

In [ ]:
my_traj = traj_collection.trajectories[13]
m = my_traj.explore(style_kwds={'weight': 5}, name="My trajectory", color="blue")
my_traj.df.explore(m=m, style_kwds={'weight': 5, 'color': 'red'}, name="My trajectory points")
folium.LayerControl().add_to(m)
m

In [ ]:
my_traj_smoothed = mpd.OutlierCleaner(my_traj).clean(v_max=100, units=("km", "h"))

In [ ]:
m = my_traj_smoothed.explore(style_kwds={'weight': 5}, name="My trajectory", color="blue")
my_traj_smoothed.df.explore(m=m, style_kwds={'weight': 5, 'color': 'red'}, name="My trajectory points")
folium.LayerControl().add_to(m)
m

### ObservationGapSplitter

Split the trajectory where then are no observations for at least two minutes:



In [ ]:
split = mpd.ObservationGapSplitter(my_traj_smoothed).split(gap=timedelta(minutes=2))
split

In [ ]:
m = split.explore(style_kwds={'weight': 5}, column="TAXI_ID",  name="My trajectory", cmap="bwr")
my_traj_smoothed.df.explore(m=m, style_kwds={'weight': 5, 'color': 'red'}, name="My trajectory points")
folium.LayerControl().add_to(m)
m